In [22]:
import sys
sys.path.append('../ames') # path the the directory

import numpy as np
import pandas as pd
import statsmodels.api as sm
from statsmodels.api import OLS
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
import re
import itertools
import statistics
%matplotlib inline 

In [66]:
import numpy as np
import pandas as pd
import sys
sys.path.append('../ames')
import config

def clean(housing_location, save_to_disk=True, output_location=config.HOUSING_PICKLE):

    housing = pd.read_csv(housing_location)

    # Drop index column
    housing.drop('Unnamed: 0',axis=1, inplace=True)

    # Drop duplicate record(s)
    housing = housing.loc[~housing.duplicated(),:]

    # Impute NAs with zeros (Continuous Variable)
    continuous_var = ['BsmtFinSF1','BsmtFinSF2','BsmtFullBath','BsmtHalfBath',
                    'BsmtUnfSF','GarageArea','GarageCars',
                    'LotFrontage','MasVnrArea','TotalBsmtSF']
    for var in continuous_var:
        housing[var].fillna(0, inplace=True)

    # Impute NAs with None (Qualitative Variable)
    housing['BsmtQual'].fillna('None', inplace=True)

    # TODO: Additional cleaning

    if save_to_disk:
        # Save to a Pickle file for ease of transport
        housing.to_pickle(output_location)

    return housing


def add_features(housing, save_to_disk=True, output_location=config.HOUSING_PICKLE):
    """
    This function will add features to the housing dataframe.
    Input: housing (pandas dataframe)
    Output: pandas dataframe
    """

    housing['TotalLivingArea'] = housing['GrLivArea'] + housing['TotalBsmtSF'] - housing['BsmtUnfSF']
    housing['UnusedLotSize'] = housing['LotArea'] - housing['1stFlrSF']

    housing['HasPool'] = np.where(housing['PoolArea']>0, 1, 0)
    housing['HasBsmt'] = np.where(housing['BsmtQual']=='None', 0, 1)

    housing['Toilets'] = housing['HalfBath'] + housing['FullBath'] + housing['BsmtFullBath'] + housing['BsmtHalfBath']
    housing['Showers'] = housing['FullBath'] + housing['BsmtFullBath']

    housing['DecadeBuilt'] = housing['YearBuilt'].apply(lambda x: (x//10 * 10))
    housing['DecadeRemodel'] = housing['YearBuilt'].apply(lambda x: (x//10 * 10))
    
    maxYear = max(housing['YearBuilt'])
    housing['HouseAge'] = maxYear - housing['YearBuilt'] + 1
    housing['HouseAgeSq'] = housing['HouseAge'] ** 2
    housing['HouseAgeLog'] = np.log(housing['HouseAge'])

    housing['UpDownRatio'] = housing['2ndFlrSF']/housing['1stFlrSF'] # Ratio of area upstairs to area downstairs

    if save_to_disk:
        # Save to a Pickle file for ease of transport
        housing.to_pickle(output_location)

    return housing

def dummify(housing, dict_of_dummy_var, drop_first=False):
    """
    Purpose: Dummify categorical variables
    Inputs: housing = DataFrame
            dict_of_dummy_var = Dict with name of categorical variable as key and prefix as value
            drop_first = Boolean to determine whether to drop the first categorical dummy
    Output: DataFrame which should be joined with another dataframe to get all the features
    """
    X = pd.DataFrame()

    if not dict_of_dummy_var:
        return X

    for var, prefix in dict_of_dummy_var.items():
        dummy_df = pd.get_dummies(housing[var], prefix=prefix, drop_first=drop_first)
        X = pd.concat([X, dummy_df], axis=1)

    return X

In [157]:
from statsmodels.stats.outliers_influence import variance_inflation_factor 
housing=clean("Ames_Housing_Price_Data.csv")
housing=add_features(housing)
dict_of_dummy_var={'Neighborhood':'Nbhd','LotConfig':'LC','SaleCondition':'SC','BldgType':'BT','BsmtQual':'BQ'}
housing2=dummify(housing,dict_of_dummy_var, drop_first=False)
# the independent variables set 
X = housing[['LotFrontage','UnusedLotSize','HouseAge','OverallQual','OverallCond',
                    'TotalLivingArea','Toilets','Showers','UpDownRatio','GarageArea','HasPool','HasBsmt']] 
X=pd.concat([X,housing2],axis=1) 
cols=[1]
#X=X.drop(X.columns[cols], axis=1,inplace=True)
# VIF dataframe 
vif_data = pd.DataFrame() 
vif_data["feature"] = X.columns 
  
# calculating VIF for each feature 
vif_data["VIF"] = [variance_inflation_factor(X.values, i) 
                          for i in range(len(X.columns))] 
  
print(vif_data)

/opt/anaconda3/lib/python3.7/site-packages/statsmodels/stats/outliers_influence.py:193: RuntimeWarning: divide by zero encountered in double_scalars
  vif = 1. / (1. - r_squared_i)


            feature       VIF
0       LotFrontage  1.316544
1     UnusedLotSize  1.363145
2          HouseAge  6.403690
3       OverallQual  3.082397
4       OverallCond  1.348015
5   TotalLivingArea  3.087252
6           Toilets  5.775421
7           Showers  4.238196
8       UpDownRatio  1.982225
9        GarageArea  1.886038
10          HasPool  1.030039
11          HasBsmt       inf
12     Nbhd_Blmngtn       inf
13     Nbhd_Blueste       inf
14      Nbhd_BrDale       inf
15     Nbhd_BrkSide       inf
16     Nbhd_ClearCr       inf
17     Nbhd_CollgCr       inf
18     Nbhd_Crawfor       inf
19     Nbhd_Edwards       inf
20     Nbhd_Gilbert       inf
21      Nbhd_Greens       inf
22     Nbhd_GrnHill       inf
23      Nbhd_IDOTRR       inf
24     Nbhd_Landmrk       inf
25     Nbhd_MeadowV       inf
26     Nbhd_Mitchel       inf
27       Nbhd_NAmes       inf
28     Nbhd_NPkVill       inf
29      Nbhd_NWAmes       inf
30     Nbhd_NoRidge       inf
31     Nbhd_NridgHt       inf
32     Nbh

In [156]:
from statsmodels.stats.outliers_influence import variance_inflation_factor 
housing=clean("Ames_Housing_Price_Data.csv")
housing=add_features(housing)
dict_of_dummy_var={'Neighborhood':'Nbhd','LotConfig':'LC','SaleCondition':'SC','BldgType':'BT','BsmtQual':'BQ'}
housing2=dummify(housing,dict_of_dummy_var, drop_first=False)
# the independent variables set 
X = housing[['LotFrontage','UnusedLotSize','HouseAgeSq','OverallQual','OverallCond',
                    'TotalLivingArea','Toilets','Showers','UpDownRatio','GarageArea','HasPool','HasBsmt']] 
X=pd.concat([X,housing2],axis=1) 
cols=[1]
#X=X.drop(X.columns[cols], axis=1,inplace=True)
# VIF dataframe 
vif_data = pd.DataFrame() 
vif_data["feature"] = X.columns 
  
# calculating VIF for each feature 
vif_data["VIF"] = [variance_inflation_factor(X.values, i) 
                          for i in range(len(X.columns))] 
  
print(vif_data)

/opt/anaconda3/lib/python3.7/site-packages/statsmodels/stats/outliers_influence.py:193: RuntimeWarning: divide by zero encountered in double_scalars
  vif = 1. / (1. - r_squared_i)


            feature       VIF
0       LotFrontage  1.317306
1     UnusedLotSize  1.349919
2        HouseAgeSq  3.942720
3       OverallQual  3.022867
4       OverallCond  1.321825
5   TotalLivingArea  3.070811
6           Toilets  5.780927
7           Showers  4.254495
8       UpDownRatio  2.013932
9        GarageArea  1.877594
10          HasPool  1.030084
11          HasBsmt       inf
12     Nbhd_Blmngtn       inf
13     Nbhd_Blueste       inf
14      Nbhd_BrDale       inf
15     Nbhd_BrkSide       inf
16     Nbhd_ClearCr       inf
17     Nbhd_CollgCr       inf
18     Nbhd_Crawfor       inf
19     Nbhd_Edwards       inf
20     Nbhd_Gilbert       inf
21      Nbhd_Greens       inf
22     Nbhd_GrnHill       inf
23      Nbhd_IDOTRR       inf
24     Nbhd_Landmrk       inf
25     Nbhd_MeadowV       inf
26     Nbhd_Mitchel       inf
27       Nbhd_NAmes       inf
28     Nbhd_NPkVill       inf
29      Nbhd_NWAmes       inf
30     Nbhd_NoRidge       inf
31     Nbhd_NridgHt       inf
32     Nbh

In [145]:
X.head()

,LotFrontage,UnusedLotSize,HouseAge,HouseAgeSq,OverallQual,OverallCond,TotalLivingArea,Toilets,Showers,UpDownRatio,...,BT_2fmCon,BT_Duplex,BT_Twnhs,BT_TwnhsE,BQ_Ex,BQ_Fa,BQ_Gd,BQ_None,BQ_Po,BQ_TA
0,0.0,7034,72,5184,6,6,1094.0,2.0,2.0,0.000000,...,0,0,0,0,0,0,0,0,0,1
1,42.0,3186,27,729,5,5,1994.0,3.0,3.0,0.000000,...,0,0,0,1,0,0,1,0,0,0
2,60.0,5059,81,6561,5,9,1738.0,1.0,1.0,0.000000,...,0,0,0,0,0,0,0,0,0,1
3,80.0,7429,111,12321,4,8,1039.0,1.0,1.0,0.449093,...,0,0,0,0,0,1,0,0,0,0
4,70.0,7590,10,100,8,6,2308.0,4.0,3.0,1.055556,...,0,0,0,0,0,0,1,0,0,0
